In [23]:
import pandas as pd
import json

# Leer el JSON desde la ruta completa
with open(r"C:\Users\pablo\proyecto\comparador\alquiler_comparador_limpio.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Detectar la lista principal dentro del JSON
if isinstance(data, dict):
    for key, value in data.items():
        if isinstance(value, list):
            df = pd.DataFrame(value)
            break
else:
    df = pd.DataFrame(data)

# Verificamos si contiene la columna "link"
if "link" not in df.columns:
    raise ValueError("No se encontró la columna 'link' en el JSON.")

urls = df["link"].dropna().unique().tolist()[:5]
print(urls)


['https://www.pisos.com/alquilar/piso-la_finca-49178402159_280500/', 'https://www.pisos.com/alquilar/piso-galapagar_la_navata28420-43338541420_100500/', 'https://www.pisos.com/alquilar/casa_unifamiliar-las_matas_los_penascales-29189878080_280500/', 'https://www.pisos.com/alquilar/piso-rios_rosas28003-52592214693_104700/', 'https://www.pisos.com/alquilar/piso-zona_avenida_de_europa28224-45068829832_101800/']


In [37]:
import time
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

# Aceptar cookies si aparecen
def aceptar_cookies(driver):
    try:
        wait = WebDriverWait(driver, 10)
        boton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#didomi-notice-agree-button")))
        boton.click()
        print("🍪 Cookies aceptadas.")
        time.sleep(1)
    except:
        print("⚠️ Cookies ya aceptadas o no detectadas.")

# Extraer coordenadas desde el src
def extraer_coords(src):
    match = re.search(r"/(\d+_\d+_)([0-9.]+)@(-?[0-9.]+)_", src)
    if match:
        lat, lon = match.group(2), match.group(3)
        return lat, lon
    return None, None

# Leer todas las URLs desde el archivo JSON
with open(r"C:\Users\pablo\proyecto\comparador\alquiler_comparador_limpio.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Si tiene una clave externa tipo {"data": [...]}, la extraemos
if isinstance(raw_data, dict):
    for key, val in raw_data.items():
        if isinstance(val, list):
            raw_data = val
            break

df = pd.DataFrame(raw_data)
urls = df["link"].dropna().unique().tolist()

# Configuración Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Puedes quitar esto para ver el navegador
service = Service()
driver = webdriver.Chrome(service=service, options=options)

datos = []

print(f"🔍 Scrapeando coordenadas para {len(urls)} enlaces...\n")

for i, url in enumerate(urls, 1):
    print(f"{i}/{len(urls)} → {url}")
    driver.get(url)
    time.sleep(2)

    if i == 1:
        aceptar_cookies(driver)

    try:
        imagen = WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img[alt='mapa']"))
        )
        src = imagen.get_attribute("src")
        print("🧭 src:", src)
        lat, lon = extraer_coords(src)
    except:
        print("❌ No se encontró el mapa.")
        lat, lon = None, None

    datos.append({"url": url, "lat": lat, "lon": lon})

driver.quit()

# Guardar CSV
df_resultado = pd.DataFrame(datos)
df_resultado.to_csv("coordenadas_scrapeadas.csv", index=False)
print("\n✅ Coordenadas guardadas en coordenadas_scrapeadas.csv")
print(df_resultado.head())



🔍 Scrapeando coordenadas para 2616 enlaces...

1/2616 → https://www.pisos.com/alquilar/piso-la_finca-49178402159_280500/
⚠️ Cookies ya aceptadas o no detectadas.
🧭 src: https://map.imghs.net/Cache/Z2R35MG0MC1/2_350_40.4078597@-3.811254_0_1.gif
2/2616 → https://www.pisos.com/alquilar/piso-galapagar_la_navata28420-43338541420_100500/
🧭 src: https://map.imghs.net/Cache/Z2R35MG0MC1/2_350_40.586148562@-3.996056727_0_1.gif
3/2616 → https://www.pisos.com/alquilar/casa_unifamiliar-las_matas_los_penascales-29189878080_280500/
🧭 src: https://map.imghs.net/Cache/Z2R35MG0MC1/2_350_40.566081683@-3.897150552_0_1.gif
4/2616 → https://www.pisos.com/alquilar/piso-rios_rosas28003-52592214693_104700/
🧭 src: https://statics.imghs.net//dist/img/map_l_v1.svg
5/2616 → https://www.pisos.com/alquilar/piso-zona_avenida_de_europa28224-45068829832_101800/
🧭 src: https://map.imghs.net/Cache/Z2R12MG0MC0/2_1200_40.4325624@-3.804644_0_0.gif
6/2616 → https://www.pisos.com/alquilar/piso-justicia_chueca28004-52539451934

In [45]:
import pandas as pd
import json

with open(r"C:\Users\pablo\proyecto\comparador\alquiler_comparador_limpio.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)


data = raw_data["data"]  # ← si tu JSON tiene esa clave principal
df_json = pd.DataFrame(data)

df_coords = pd.read_csv(r"C:\Users\pablo\proyecto\coordenadas_scrapeadas.csv")

df_json["link_normalized"] = df_json["link"].str.strip().str.replace(r"\\", "", regex=True)
df_coords["url_normalized"] = df_coords["url"].str.strip()

df_merged = pd.merge(df_json, df_coords, left_on="link_normalized", right_on="url_normalized", how="left")


df_merged.drop(columns=["link_normalized", "url_normalized"], inplace=True)

df_merged.head()


,index,titulo_y,ubicacion_y,precio_num,habitaciones_num,baños_num,metros_num,superficie_construida_num,superficie_util_num,link,url,lat,lon
0,0,Piso en calle Luis Garcia Cereceda,La Finca (Pozuelo de Alarcón),8500.0,3.0,3.0,378.0,378.0,188.0,https://www.pisos.com/alquilar/piso-la_finca-4...,https://www.pisos.com/alquilar/piso-la_finca-4...,40.407860,-3.811254
1,1,Piso en calle Libertad,La Navata (Galapagar),1450.0,3.0,1.0,90.0,90.0,87.0,https://www.pisos.com/alquilar/piso-galapagar_...,https://www.pisos.com/alquilar/piso-galapagar_...,40.586149,-3.996057
2,2,Casa unifamiliar en calle XIX,Las Matas-Los Peñascales (Las Rozas de Madrid),4000.0,6.0,7.0,600.0,600.0,600.0,https://www.pisos.com/alquilar/casa_unifamilia...,https://www.pisos.com/alquilar/casa_unifamilia...,40.566082,-3.897151
3,3,Piso en calle de Ponzano,Río Rosas (Distrito Chamberí. Madrid Capital),3500.0,4.0,3.0,226.0,NaN,NaN,https://www.pisos.com/alquilar/piso-rios_rosas...,https://www.pisos.com/alquilar/piso-rios_rosas...,NaN,NaN
4,4,Piso en Zona Avenida de Europa,Zona Avenida de Europa (Pozuelo de Alarcón),1400.0,1.0,1.0,65.0,65.0,NaN,https://www.pisos.com/alquilar/piso-zona_aveni...,https://www.pisos.com/alquilar/piso-zona_aveni...,40.432562,-3.804644


In [47]:
# Guardar como archivo CSV
df_merged.to_csv("alquiler_unificado_con_coordenadas.csv", index=False)

# Confirmación
print("✅ Archivo guardado como 'alquiler_unificado_con_coordenadas.csv'")


✅ Archivo guardado como 'alquiler_unificado_con_coordenadas.csv'
